Creo un dictionary in cui riporto tutti i modelli che andrò ad analizzare

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
#from cdo import *
from funzioni import compute_dataset_weighted
from funzioni import compute_annual_mean_weighted
from funzioni import convert_dataset_celsius
from funzioni import compute_anomaly

In [52]:
#cdo = Cdo()

In [2]:
# creo un dizionario in cui andrò a mettere tutti i modelli dentro
models = {}

In [3]:
#Costruisco il dizionario models con il nome del modello, il nomde del percorso di tutti i file *.nc di ogni modello e il nome di .nc, cioè il modello remapbil

# percorso dei file model.txt e tos_nc_model.txt
name_models = '/work/users/guest/sbeltrami/prova/name_ocean_model.txt'
path_models = '/work/users/guest/sbeltrami/prova/path_ocean_model.txt'
path_remap_models = '/work/users/guest/sbeltrami/prova/path_remap_ocean_model.txt'
# Apro i file in modalità di lettura
with open(name_models, 'r') as file:
    # Leggo tutte le righe del file e salvo in una lista
    lines_name_models = file.readlines()
with open(path_models, 'r') as file:
    # Leggo tutte le righe del file e salvo in una lista
    lines_path_models = file.readlines()
with open(path_remap_models, 'r') as file:
    lines_path_remap_models = file.readlines()


# Itero su tutte le righe e organizzo i dati nel dizionario
for line_name, line_path, line_path_remap in zip(lines_name_models, lines_path_models, lines_path_remap_models):
    model_name = line_name.strip()

    # La chiave principale model_name deve essere presente nel dizionario
    if model_name not in models:
        models[model_name] = {}

    # Aggiungo i dati come sottochiavi
    models[model_name]['name'] = model_name
    models[model_name]['path'] = line_path.strip()
    models[model_name]['path remap'] = line_path_remap.strip()

In [4]:
models['TaiESM1'].items()

dict_items([('name', 'TaiESM1'), ('path', '/work/users/guest/sbeltrami/TaiESM1.nc'), ('path remap', '/work/users/guest/sbeltrami/TaiESM1_remapcon.nc')])

In [15]:
count = 0 #conteggio dei file che non si possono aprire
count_ok = 0 #conteggio dei file che apro

for name in models.keys():
    # Verifica se il file NetCDF esiste
    path_remap = models[name]['path remap']
    if os.path.exists(path_remap):
        # Se il file esiste, apri il dataset con xarray
        dataset = xr.open_dataset(path_remap)
        # Continua con le operazioni sul dataset
        #print(f"Dataset aperto con successo per il modello {name}")
        #peso il dataset
        models[name]['dataset weighted'] = compute_dataset_weighted(dataset["tos"])
        #medie annuali con dataset pesato
        if name == 'CIESM': #da modificare!!
            models[name]['dataset annual mean'] = 1
            print(models[name]['name'])
        else:
            models[name]['dataset annual mean'] = compute_annual_mean_weighted(models[name]['dataset weighted'])
            print(models[name]['name']) 
        
        #Seleziono il box del Nord Atlantico
        na_box = dataset.sel(lon=slice(260,360), lat=slice(30,70))
        #peso il dataset Nord Atlantico
        models[name]['Nord Atlantic dataset weighted'] = compute_dataset_weighted(na_box["tos"])
        #Seleziono la climatologia
        na_box_climat = na_box.sel(time=slice("1984-01", "2014-12"))
        #peso il dataset del periodo scelto per la climatologia
        models[name]['Nord Atlantic climatology weighted'] = compute_dataset_weighted(na_box_climat["tos"])
        #Calcolo l'anomalia di tos rispetto alla climatologia scelta
        models[name]['Nord Atlantic anomaly'] = compute_anomaly(models[name]['Nord Atlantic dataset weighted'],models[name]['Nord Atlantic climatology weighted'])

        count_ok = count_ok + 1
    else:
        print(f"Errore: Il file {path_remap} non esiste per il modello {name}")

        count = count + 1

TaiESM1
AWI-CM-1-1-MR
AWI-ESM-1-1-LR
BCC-CSM2-MR
BCC-ESM1
CAMS-CSM1-0
CAS-ESM2-0
FGOALS-f3-L
FGOALS-g3
CanESM5
CMCC-CM2-SR5
ACCESS-ESM1-5
ACCESS-CM2
E3SM-1-0
E3SM-1-1
E3SM-1-1-ECA
EC-Earth3
EC-Earth3-Veg
FIO-ESM-2-0
MPI-ESM-1-2-HAM
INM-CM4-8
INM-CM5-0
IPSL-CM6A-LR
MIROC6
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
GISS-E2-1-G
GISS-E2-1-G-CC
GISS-E2-1-H
CESM2
CESM2-FV2
CESM2-WACCM
CESM2-WACCM-FV2
NorCPM1
NorESM2-LM
NorESM2-MM
GFDL-CM4
GFDL-ESM4
NESM3
SAM0-UNICON
CIESM


KeyError: cftime.DatetimeNoLeap(1984, 1, 1, 0, 0, 0, 0, has_year_zero=True)

In [13]:
len(models)

43

Nord Atlantico anomalie

Dataset ESA-CCI

In [11]:
#ESA-CCI box Nord Atlantico
dataset_ESA_na=xr.open_dataset('/work/users/guest/sbeltrami/prova/ESA-CCI_SST_remap.nc').sel(lon=slice(260,360), lat=slice(30,70)) #seleziono lon lat del Nord Atlantico
dataset_ESA_na_celsius = convert_dataset_celsius(dataset_ESA_na["analysed_sst"]) #converto in °C
#dataset_ESA_na_celsius.isel(time=0).plot()
#dataset pesato
dataset_ESA_weighted_na = compute_dataset_weighted(dataset_ESA_na_celsius)
#Scelgo periodo su cui calcolare la climatologia
esa_na_1984_2014 = dataset_ESA_na.sel(time=slice("1984-01", "2014-12"))
esa_na_1984_2014_deg = convert_dataset_celsius(esa_na_1984_2014["analysed_sst"])
esa_na_1984_2014_deg = convert_dataset_celsius(esa_na_1984_2014["analysed_sst"])
#peso il dataset del periodo 1984-2014
#esa_na_1984_2014_weighted = compute_dataset_weighted(esa_na_1984_2014_deg)
annual_mean_weighted_30 = compute_dataset_weighted(esa_na_1984_2014_deg)
#Calcolo l'anomalia di tos rispetto al periodo di riferimento 1984-2014
anomaly_esa = compute_anomaly(dataset_ESA_weighted_na,esa_na_1984_2014_weighted)
plot
anomaly_esa.plot()
plt.suptitle('Anomalia Nord Atlantico ESA rispetto alla media climatologica 1984 - 2014')

TypeError: unsupported operand type(s) for -: 'DataArrayWeighted' and 'float'

Modelli Nord Atlantico

In [ ]:
#box Nord Atlantico
for name in models.keys():
    #Seleziono il box Nord Atlantico
    na_box = 
    models[name][]
dataset_ESA_na=xr.open_dataset('/work/users/guest/sbeltrami/prova/ESA-CCI_SST_remap.nc').sel(lon=slice(260,360), lat=slice(30,70)) #seleziono lon lat del Nord Atlantico
dataset_ESA_na_celsius = convert_dataset_celsius(dataset_ESA_na["analysed_sst"]) #converto in °C
#dataset_ESA_na_celsius.isel(time=0).plot()
#dataset pesato
dataset_ESA_weighted_na = compute_dataset_weighted(dataset_ESA_na_celsius)
#Scelgo periodo su cui calcolare la climatologia
esa_na_1984_2014 = dataset_ESA_na.sel(time=slice("1984-01", "2014-12"))
esa_na_1984_2014_deg = convert_dataset_celsius(esa_na_1984_2014["analysed_sst"])
esa_na_1984_2014_deg = convert_dataset_celsius(esa_na_1984_2014["analysed_sst"])
#peso il dataset del periodo 1984-2014
#esa_na_1984_2014_weighted = compute_dataset_weighted(esa_na_1984_2014_deg)
annual_mean_weighted_30 = compute_dataset_weighted(esa_na_1984_2014_deg)
#Calcolo l'anomalia di tos rispetto al periodo di riferimento 1984-2014
anomaly_esa = compute_anomaly(dataset_ESA_weighted_na,esa_na_1984_2014_weighted)
plot
anomaly_esa.plot()
plt.suptitle('Anomalia Nord Atlantico ESA rispetto alla media climatologica 1984 - 2014')

In [ ]:
tos = xr.open_mfdataset(paths="/archive/paolo/cmip6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/historical/ocean/Omon/r1i1p1f1/tos/*.nc")

In [ ]:
new_tos= xr.open_dataset('/work/users/guest/sbeltrami/tostest.nc') #tostest.nc contiene tutti i dati concatenati in tos, usando il comando: cdo remapbil,r180x90 -cat *.nc /work/users/guest/sbeltrami/tostest.nc

prova con soli 3 modelli

In [5]:
models = { 'Ec-Earth3':
                    {'path': '/work/users/guest/sbeltrami/tostest.nc',
                     },
            'IPSL-CM6A-LR':
                    {'path': '/work/users/guest/sbeltrami/tostest_ipsl.nc',
                     },
            'MPI-ESM1-2-HR':
                    {'path': '/work/users/guest/sbeltrami/tostest_mpi.nc',
                     }
         }

In [6]:
for name in models.keys():
    #apertura modelli
    dataset = xr.open_dataset(models[name]['path'])
    #peso il dataset
    models[name]['dataset weighted'] = compute_dataset_weighted(dataset["tos"])
    #medie annuali con dataset pesato
    models[name]['annual mean'] = compute_annual_mean_weighted(models[name]['dataset weighted'])


In [9]:
models['Ec-Earth3'].keys()

dict_keys(['path', 'dataset weighted', 'annual mean'])

In [10]:
models['Ec-Earth3'].items()